# Import Solidity Source Code

In [ ]:
f = open("output.sol", "r")
file_content = f.read()
f.close()

print(file_content)

In [81]:
# For testing
file_content = """
pragma solidity ^0.4.24;

contract something {
  uint public coolade;
  address public checker;
  address public owner;
  mapping(address => bool) public whitelist;
  int public more;
  int256 public more3;
  int8 more2;
  uint[4] coolade2;
  int[][2] coolade3;
  address[] fishe;
  string[] arr;

  constructor() public {
    owner = msg.sender;
  }

  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }

  function setChecker(address _checker) public onlyOwner {
    checker = _checker;
  }

  function approve(address _wallet) public onlyOwner {
    whitelist[_wallet] = true;
  }
}
"""

print(file_content)


pragma solidity ^0.4.24;

contract something {
  uint public coolade;
  address public checker;
  address public owner;
  mapping(address => bool) public whitelist;
  int public more;
  int256 public more3;
  int8 more2;
  uint[4] coolade2;
  int[][2] coolade3;
  address[] fishe;
  string[] arr;

  constructor() public {
    owner = msg.sender;
  }

  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }

  function setChecker(address _checker) public onlyOwner {
    checker = _checker;
  }

  function approve(address _wallet) public onlyOwner {
    whitelist[_wallet] = true;
  }
}



In [4]:
import re

# Get contract version
#version = re.search('pragma solidity \^?(.*);', file_content).group(1)
# Get contract names
#name = re.findall('contract (.*) {', file_content)

# Utility functions
def swapPositionList(lst, pos1, pos2):
  lst[pos1], lst[pos2] = lst[pos2], lst[pos1]
  return lst

def swapPositionDict(dct, key1, key2):
  dct[key1], dct[key2] = dct[key2], dct[key1]
  return dct

def bitToByte(bit):
  return bit/8

In [86]:
### Process:
# Seperate source code to each lines -> list
# Scan and mark each lines based on its characteristics -> dict
# Optimize source code
# Combine all source code lines back

### Variable unpacking
# Scan for all variable islands
# Sort each variable islands from big to small byte size

### Seperate source code to each lines -> list
code_lines = file_content.split("\n")
code_metadata = {}

# Variables for detecting issues
all_vars = []  # Stores all islands of vars. Content: dict of byte sizes group
current_island_var = -1
last_index_var = -1  # To detect when the islands cut off.  if detected var > last_index_var+1
sorted_var = []

# 0: Symbol opening/closing, 1: major opener, 2: data types, 3: visibility
tags = {0: "{",
        1: "}",
        100: "contract ",
        101: " function ",
        200: " bool ",
        201: " int", #[8-256]?
        202: " uint", #[8-256]?
        203: " bytes", #[1-32]?
        204: " address",
        205: " mapping\(",
        206: " string",
        301: " public ",
        }  # token: tag

def add_variable_island(byte_size, index):
  global last_index_var
  global current_island_var
  global all_vars

  # Start new island
  if last_index_var+1 < index:
    current_island_var += 1
    all_vars.append({})
    all_vars[current_island_var]["metadata"] = {}
    all_vars[current_island_var]["metadata"]["start_index"] = index
    all_vars[current_island_var]["metadata"]["total_line"] = 0
    all_vars[current_island_var]["metadata"]["byte_sizes"] = set()

  # Add to island var
  all_vars[current_island_var].setdefault(byte_size, []).append(index)
  all_vars[current_island_var]["metadata"]["total_line"] += 1
  all_vars[current_island_var]["metadata"]["byte_sizes"].add(byte_size)
  last_index_var = index

### Scan and mark each lines based on its characteristics -> dict
for index, value in enumerate(code_lines):
  metadata = [value]
  for token in tags:

    # https://pynative.com/python-regex-pattern-matching/
    # Check each code line for tag tokens
    if re.search(tags[token], value) != None:
      metadata += [token]

      ## Scan variable islands
      if token == 200:  # bool
        print("bool detected on index", index)

        # Check byte size
        size = re.search(r' bool(.*?) ', value).group(1)
        if size == "":
          byte_size = 1
        elif size.isdecimal() == False:
          byte_size = 32
        add_variable_island(byte_size, index)

      elif token == 201:  # int
        print("int detected on index", index)

        # Check byte size
        size = re.search(r' int(.*?) ', value).group(1)
        if size.isdecimal() == False:
          size = 256
        byte_size = int(size)//8
        add_variable_island(byte_size, index)

      elif token == 202:  # uint
        print("uint detected on index", index)

        # Check byte size
        size = re.search(r' uint(.*?) ', value).group(1)
        if size.isdecimal() == False:
          size = 256
        byte_size = int(size)//8
        add_variable_island(byte_size, index)

      elif token == 203:  # bytes
        print("bytes detected on index", index)

        # Check byte size
        byte_size = re.search(r' bytes(.*?) ', value).group(1)
        if byte_size.isdecimal() == False:
          byte_size = 32
        add_variable_island(int(byte_size), index)

      elif token == 204:  # address
        print("address detected on index", index)

        # Check byte size
        size = re.search(r' address(.*?) ', value).group(1)
        if size == "":
          byte_size = 20
        elif size.isdecimal() == False:
          byte_size = 32
        add_variable_island(byte_size, index)

      elif token == 205:  # mapping
        print("mapping detected on index", index)
        add_variable_island(32, index)

      elif token == 206:  # string
        print("string detected on index", index)
        add_variable_island(32, index)


  code_metadata[index] = metadata

### Optimize source code
print("")
# Variable Packing
for index, value in enumerate(all_vars):
  # Sort the variables based on their byte size
  sorted_var = []
  for i in sorted(value["metadata"]["byte_sizes"], reverse=True):
    sorted_var.extend(value[i])

  print("Sorted vars:", sorted_var)

  # Organize source code to be sorted
  for idx, val in enumerate(sorted_var):  # Copy the source code in order
    sorted_var[idx] = code_lines[val]
  for idx, val in enumerate(sorted_var):  # Paste the source code to the real list
    code_lines[value["metadata"]["start_index"]+idx] = val

print("")
#for index, value in enumerate(code_lines):
#  pass

### Combine all source code lines back
code_optimized = '\n'.join(code_lines)

print(all_vars)
print(code_optimized)
code_metadata

uint detected on index 4
address detected on index 5
address detected on index 6
mapping detected on index 7
int detected on index 8
int detected on index 9
int detected on index 10
uint detected on index 11
int detected on index 12
address detected on index 13
string detected on index 14

Sorted vars: [4, 7, 8, 9, 11, 12, 13, 14, 5, 6, 10]

[{'metadata': {'start_index': 4, 'total_line': 11, 'byte_sizes': {32, 1, 20}}, 32: [4, 7, 8, 9, 11, 12, 13, 14], 20: [5, 6], 1: [10]}]

pragma solidity ^0.4.24;

contract something {
  uint public coolade;
  mapping(address => bool) public whitelist;
  int public more;
  int256 public more3;
  uint[4] coolade2;
  int[][2] coolade3;
  address[] fishe;
  string[] arr;
  address public checker;
  address public owner;
  int8 more2;

  constructor() public {
    owner = msg.sender;
  }

  modifier onlyOwner() {
    require(msg.sender == owner);
    _;
  }

  function setChecker(address _checker) public onlyOwner {
    checker = _checker;
  }

  functio

{0: [''],
 1: ['pragma solidity ^0.4.24;'],
 2: [''],
 3: ['contract something {', 0, 100],
 4: ['  uint public coolade;', 202, 301],
 5: ['  address public checker;', 204, 301],
 6: ['  address public owner;', 204, 301],
 7: ['  mapping(address => bool) public whitelist;', 205, 301],
 8: ['  int public more;', 201, 301],
 9: ['  int256 public more3;', 201, 301],
 10: ['  int8 more2;', 201],
 11: ['  uint[4] coolade2;', 202],
 12: ['  int[][2] coolade3;', 201],
 13: ['  address[] fishe;', 204],
 14: ['  string[] arr;', 206],
 15: [''],
 16: ['  constructor() public {', 0, 301],
 17: ['    owner = msg.sender;'],
 18: ['  }', 1],
 19: [''],
 20: ['  modifier onlyOwner() {', 0],
 21: ['    require(msg.sender == owner);'],
 22: ['    _;'],
 23: ['  }', 1],
 24: [''],
 25: ['  function setChecker(address _checker) public onlyOwner {',
  0,
  101,
  301],
 26: ['    checker = _checker;'],
 27: ['  }', 1],
 28: [''],
 29: ['  function approve(address _wallet) public onlyOwner {', 0, 101, 301]

# The Meat (https://regexr.com/)
### https://github.com/andhikan207/SC_TestBench/blob/main/Benchmarking.ipynb
### https://pynative.com/python-regex-pattern-matching/
### https://www.regextutorial.org/regex-for-numbers-and-ranges.php

In [ ]:
from IPython.display import clear_output
import pandas as pd
import re

#File name
file_name = "dummyfile.csv"

In [ ]:
#Load smart contracts list.
df = pd.read_csv(file_name)

#Convert dataframe to list.
smartContracts = []
smartContracts2 = []
smartContracts = df['Source Code'].tolist()

#Process to add keywords to individual smart contracts.
unfinishedKW = "UNFINISHED_SC"
failedKW = "NOT_A_SC"

#Manually input keywords
numCycle1 = 1

for i in smartContracts:
  clear_output(wait = True)
  print(i + "\n")

  while True:
    print(f"{numCycle1}. Please check this smart contract. (1. Finished, 2. Unfinished, 3. Failed)")
    scCondition = input("Input a number: ")

    if scCondition == "1":
      smartContracts2.append(i)
      numCycle1 += 1
      break
    elif scCondition == "2":
      smartContracts2.append(unfinishedKW + " " + i)
      numCycle1 += 1
      break
    elif scCondition == "3":
      smartContracts2.append(failedKW + " " + i)
      numCycle1 += 1
      break
    else:
      print("\n> Wrong input. Try again!")

clear_output(wait = True)
print("Done, please proceed to the next step!")

Done, please proceed to the next step!


In [ ]:
#Choose ERC Criteria
clear_output(wait = True)
chooseERC = ""

while True:
  print("1. ERC-20")
  print("2. ERC-4626")
  print("3. ERC-3156")
  chooseERC = input("Which scenario would you choose?: ")

  if chooseERC == "1" or chooseERC == "2" or chooseERC == "3":
    break
  else:
    print("\n> Wrong input. Try again!")


#Set criteria weight
clear_output(wait = True)

cWeight1 = 0
cWeight2 = 0
cWeight3 = 0
cWeight4 = 0
cWeight5 = 0
cWeight6 = 0
cWeight7 = 0
cWeight8 = 0
cWeight9 = 0
cWeight10 = 0

criteriaDesc = ["Complete source code", "Packing variable", "Packing booleans", "Storage type", "Data type", "Variable size type", "Default value", "Functions", "Storage limiting", "Minimize on-chain data"]
cWeights = [cWeight1, cWeight2, cWeight3, cWeight4, cWeight5, cWeight6, cWeight7, cWeight8, cWeight9, cWeight10]

while True:
  totalPerc = 0
  numCycle2 = 1
  cArr = 0

  for i in criteriaDesc:
    while True:
      try:
        print(f"{numCycle2}. Please write score weight in PERCENTAGE for {i}. (Currently {totalPerc}%/100%)")
        setWeights = float(input("(Numbers only): "))

        cWeights[cArr] = 1 * (setWeights / 100)

        cArr += 1
        numCycle2 += 1
        totalPerc += setWeights
        clear_output(wait = True)
        break
      except ValueError:
        print("\n> That is not a number!")

  totalWeight = 0
  for i in cWeights:
    totalWeight += i

  if round(totalWeight) != 1.0:
    print("> Weight exceed 1.0 or below!\n")
    continue
  else:
    clear_output(wait = True)
    cArr = 0

    print("[Criteria Weight]\n")

    for i in criteriaDesc:
      print(f"{i}: {cWeights[cArr]}")
      cArr += 1

    print(f"\nTotal weight: {round(totalWeight)}\n")
    weightDone = input("Proceed? y/n: ")

    if weightDone == "n":
      clear_output(wait = True)
      continue
    else:
      break

clear_output(wait = True)
print("Done, please proceed to the next step!")

Done, please proceed to the next step!


In [ ]:
#Benchmarking
def benchmarkProgram():
  #REGEX PATTERNS
  criteria1_UFSC = unfinishedKW #Unfinished Source Code
  criteria2_ZONK = failedKW #Failed Source Code
  notPacked = r"(uint|uint(256))+ public [A-Za-z0-9]+ = [0-9]+;(\n)+\s+uint(8|16|32|128)" #Packing variable
  isBoolean = r"(uint|uint[0-9])+\([0-9]\)" #Packing boolean

  #Storage Type
  possibleC1_V1 = "uint"
  possibleC1_V2 = "uint256"
  pat7 = r"uint(?!(?:256))\d+ public" # uint*

  pat1 = r"mapping\([^)]*\) public [A-Za-z0-9]+;" #Mapping
  pat3 = r"[A-Za-z0-9]+\[\] public [A-Za-z0-9]+;" #Array

  fixA1 = r"(uint|string|int)\[[0-9]+\]" #Fixed
  pat6 = r"[A-Za-z0-9]+\[\]" #Dynamic array

  pat4 = r"public [A-Za-z0-9]+ = 0;" #Default value

  #Functions
  extfunc = r"function [A-Za-z0-9]+\([^)]*\) external" #External function
  pubfunc = r"function [A-Za-z0-9]+\([^)]*\) public" #Pub funct

  def findCriteria(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

  def checkCriteria(pattern):
    return re.compile(pattern, re.IGNORECASE).search


  #Init settings
  benchmark_score = 0.0

  indivExcessive = []
  indivScores = []

  perfect_score = 100.0
  partial_score = 50.0
  no_score = 0.0

  #Information Tracking
  completeSC1 = 0
  completeSC2 = 0
  completeSC3 = 0
  storageType1 = 0
  storageType2 = 0
  storageType3 = 0
  packingVar1 = 0
  packingVar2 = 0
  packingBool1 = 0
  packingBool2 = 0
  defaultVal1 = 0
  defaultVal2 = 0
  dataType1 = 0
  dataType2 = 0
  dataType3 = 0
  sizeType1 = 0
  sizeType2 = 0
  functionType1 = 0
  functionType2 = 0
  functionType3 = 0
  metCriteria = 0
  exceedData = 0

  numCycle5 = 0
  for sc in smartContracts2:
    excessiveData = 0
    temp_score = 0.0

    #Complete SC
    if findCriteria(criteria2_ZONK)(sc):
      completeSC1 += 1
      temp_score += no_score
    elif findCriteria(criteria1_UFSC)(sc):
      completeSC2 += 1
      temp_score += (partial_score * cWeights[0])
    else:
      completeSC3 += 1
      temp_score += (perfect_score * cWeights[0])

    #Packing Variables
    if checkCriteria(notPacked)(sc):
      packingVar1 += 1
      temp_score += no_score
    else:
      packingVar2 += 1
      temp_score += (perfect_score * cWeights[1])

    #Packing Boolean
    if checkCriteria(isBoolean)(sc):
      packingBool1 += 1
      temp_score += (perfect_score * cWeights[2])
    else:
      packingBool2 += 1
      temp_score += no_score

    #Storage type
    if checkCriteria(pat7)(sc) and findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc): # Variety uint
      storageType1 += 1
      temp_score += (partial_score * cWeights[3])
    elif findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc) and not checkCriteria(pat7)(sc): # uint256
      storageType2 += 1
      temp_score += no_score
    elif checkCriteria(pat7)(sc) and not findCriteria(possibleC1_V1)(sc) or findCriteria(possibleC1_V2)(sc): # uint*
      storageType3 += 1
      temp_score += (perfect_score * cWeights[3])

    #Mapping vs array
    if checkCriteria(pat1)(sc) and checkCriteria(pat3)(sc): # Mapping & Array
      dataType1 += 1
      temp_score += (partial_score * cWeights[4])
    elif checkCriteria(pat1)(sc) and not checkCriteria(pat3)(sc): # Mapping
      dataType2 += 1
      temp_score += (perfect_score * cWeights[4])
    elif checkCriteria(pat3)(sc) and not checkCriteria(pat1)(sc): # Array
      dataType3 += 1
      temp_score += no_score

    #Variable size type
    if checkCriteria(fixA1)(sc): # Fixed
      sizeType1 += 1
      temp_score += (perfect_score * cWeights[5])
    else: # Dynamic
      sizeType2 += 1
      temp_score += no_score

    #Default value
    if checkCriteria(pat4)(sc): # Initialized value
      defaultVal1 += 1
      temp_score += no_score
    else:
      defaultVal2 += 1
      temp_score += (perfect_score * cWeights[6]) # Default

    #Function
    if checkCriteria(extfunc)(sc) and checkCriteria(pubfunc)(sc): #mixed function
      functionType1 += 1
      temp_score += (partial_score * cWeights[7])
    elif checkCriteria(extfunc)(sc) and not checkCriteria(pubfunc)(sc): # only external
      functionType2 += 1
      temp_score += (perfect_score * cWeights[7])
    elif checkCriteria(pubfunc)(sc) and not checkCriteria(extfunc)(sc): # only public
      functionType3 += 1
      temp_score += no_score

    #Limit Storage & On-Chain data
    if chooseERC == "1":
      #ERC20 Criterias
      erc20_1 = "function name\([^)]*\)"
      erc20_2 = "function symbol\([^)]*\)"
      erc20_3 = "function decimal\([^)]*\)"
      erc20_4 = "function totalSupply\([^)]*\)"
      erc20_5 = "function balanceOf\([^)]*\)"
      erc20_6 = "function allowance\([^)]*\)"
      erc20_7 = "function transfer\([^)]*\)"
      erc20_8 = "function approve\([^)]*\)"
      erc20_9 = "function transferFrom\([^)]*\)"
      erc20_10 = "event Transfer\([^)]*\)"
      erc20_11 = "event Approval\([^)]*\)"

      offchains1 = "function (?!totalSupply|_totalSupply|balanceOf|_balanceOf|allowance|_allowance|transfer|_transfer|approve|_approve|transferFrom|_transferFrom|decimal|_decimal|symbol|_symbol|name|_name)\w+\([^)]*\)"
      offchains2 = "event (?!Approval|Transfer)\w+\([^)]*\)"

      tS = 0
      regexListSC1 = [erc20_1, erc20_2, erc20_3, erc20_4, erc20_5, erc20_6, erc20_7, erc20_8, erc20_9, erc20_10, erc20_11]

      erc20score = 100 / len(regexListSC1)
      ERC20criteriaMet = 0

      for testCase in regexListSC1:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc20score
          ERC20criteriaMet += 1

      if ERC20criteriaMet == len(regexListSC1):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])

    elif chooseERC == "2":
      #ERC4626 Criterias
      erc4626_1 = "function asset\([^)]*\)"
      erc4626_2 = "function totalAssets\([^)]*\)"
      erc4626_3 = "function convertToShares\([^)]*\)"
      erc4626_4 = "function convertToAssets\([^)]*\)"
      erc4626_5 = "function maxDeposit\([^)]*\)"
      erc4626_6 = "function previewDeposit\([^)]*\)"
      erc4626_7 = "function deposit\([^)]*\)"
      erc4626_8 = "function maxMint\([^)]*\)"
      erc4626_9 = "function previewMint\([^)]*\)"
      erc4626_10 = "function mint\([^)]*\)"
      erc4626_11 = "function maxWithdraw\([^)]*\)"
      erc4626_12 = "function previewWithdraw\([^)]*\)"
      erc4626_13 = "function withdraw\([^)]*\)"
      erc4626_14 = "function maxRedeem\([^)]*\)"
      erc4626_15 = "function previewRedeem\([^)]*\)"
      erc4626_16 = "function redeem\([^)]*\)"
      erc4626_17 = "event Deposit\([^)]*\)"
      erc4626_18 = "event Withdraw\([^)]*\)"

      offchains1 = "function (?!asset|totalAssets|convertToShares|convertToAssets|maxDeposit|previewDeposit|deposit|maxMint|previewMint|mint|maxWithdraw|previewWithdraw|withdraw|maxRedeem|previewRedeem|redeem)\w+\([^)]*\)"
      offchains2 = "event (?!Deposit|Withdraw)\w+\([^)]*\)"

      tS = 0
      regexListSC2 = [erc4626_1, erc4626_2, erc4626_3, erc4626_4, erc4626_5, erc4626_6, erc4626_7, erc4626_8, erc4626_9, erc4626_10, erc4626_11, erc4626_12, erc4626_13, erc4626_14, erc4626_15, erc4626_16, erc4626_17, erc4626_18]

      erc4626score = 100 / len(regexListSC2)
      ERC4626criteriaMet = 0

      for testCase in regexListSC2:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc4626score
          ERC4626criteriaMet += 1

      if ERC4626criteriaMet == len(regexListSC2):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])

    elif chooseERC == "3":
      erc3156_1 = "function maxFlashLoan\([^)]*\)"
      erc3156_2 = "function flashFee\([^)]*\)"
      erc3156_3 = "function flashLoan\([^)]*\)"
      erc3156_4 = "function onFlashLoan\([^)]*\)"

      offchains1 = "function (?!maxFlashLoan|flashFee|flashLoan|onFlashLoan)\w+\([^)]*\)"
      offchains2 = "event [A-Za-z0-9]+\([^)]*\)"

      tS = 0
      regexListSC3 = [erc3156_1, erc3156_2, erc3156_3, erc3156_4]

      erc3156score = 100 / len(regexListSC3)
      ERC3156criteriaMet = 0

      for testCase in regexListSC3:
        i = checkCriteria(testCase)(sc)

        if i:
          tS += erc3156score
          ERC3156criteriaMet += 1

      if ERC3156criteriaMet == len(regexListSC3):
        metCriteria += 1

      #Check on Chains
      checkOffChain1 = len(re.findall(offchains1, sc, re.IGNORECASE))
      checkOffChain2 = len(re.findall(offchains2, sc, re.IGNORECASE))

      excessiveData = excessiveData + (checkOffChain1 + checkOffChain2)
      offChainScore = 100 - ((checkOffChain1 + checkOffChain2) * 10)

      if offChainScore < 0:
        offChainScore = 0
        exceedData += 1

      temp_score += (tS * cWeights[8])
      temp_score += (offChainScore * cWeights[9])


    indivExcessive.append(excessiveData)
    indivScores.append(round(temp_score, 2))
    numCycle5 += 1

    #Set final score
    benchmark_score += temp_score

  #Information Output
  print(f"[BENCHMARKING {numCycle5} SMART CONTRACTS RESULT]\n")
  print("Individual excessive data count:")
  print(', '.join([str(x) for x in indivExcessive]))
  print("Individual score:")
  print(', '.join([str(x) for x in indivScores]))
  print("\n")

  print("Complete source code")
  print("==============================")
  print(f"Complete SC: {completeSC3}")
  print(f"Unfinished SC: {completeSC2}")
  print(f"Failed SC: {completeSC1}\n")
  print("Packing variable")
  print("==============================")
  print(f"Packed Variable: {packingVar2}")
  print(f"Unpacked Variable: {packingVar1}\n")
  print("Packing boolean")
  print("==============================")
  print(f"Packed Bool: {packingBool1}")
  print(f"Unpacked Bool: {packingBool2}\n")
  print("Storage type")
  print("==============================")
  print(f"uint*: {storageType3}")
  print(f"Mixed storage: {storageType1}")
  print(f"uint256: {storageType2}\n")
  print("Data type")
  print("==============================")
  print(f"Only mapping: {dataType2}")
  print(f"Mixed: {dataType1}")
  print(f"Only array: {dataType3}\n")
  print("Variable size type")
  print("==============================")
  print(f"Fixed: {sizeType1}")
  print(f"Dynamic: {sizeType2}\n")
  print("Default value")
  print("==============================")
  print(f"Default Value: {defaultVal2}")
  print(f"Initialized: {defaultVal1}\n")
  print("Functions")
  print("==============================")
  print(f"External: {functionType2}")
  print(f"Mixed: {functionType1}")
  print(f"Public: {functionType3}\n")
  print("Storage limiting")
  print("==============================")
  print(f"Met storage standard: {metCriteria}\n")
  excessiveData /= 20
  print("Minimize on-chain data")
  print("==============================")
  print(f"Number of SC that exceed maximum on-chain data: {exceedData}\n")

  #Calculate average score
  print("============================================================")
  print(f"Total score: {round(benchmark_score, 2)}")
  benchmark_score /= len(smartContracts)
  print(f"Average score: {round(benchmark_score, 2)}")

benchmarkProgram()

[BENCHMARKING 5 SMART CONTRACTS RESULT]

Individual excessive data count:
4, 2, 0, 0, 1
Individual score:
57.36, 48.0, 58.64, 49.55, 55.82


Complete source code
Complete SC: 5
Unfinished SC: 0
Failed SC: 0

Packing variable
Packed Variable: 5
Unpacked Variable: 0

Packing boolean
Packed Bool: 0
Unpacked Bool: 5

Storage type
uint*: 0
Mixed storage: 4
uint256: 1

Data type
Only mapping: 4
Mixed: 1
Only array: 0

Variable size type
Fixed: 0
Dynamic: 5

Default value
Default Value: 3
Initialized: 2

Functions
External: 1
Mixed: 1
Public: 3

Storage limiting
Met storage standard: 0

Minimize on-chain data
Number of SC that exceed maximum on-chain data: 0

Total score: 269.36
Average score: 53.87
